In [17]:
import pickle
from GraphEmbeddingEncoder import GraphEmbeddingEncoder
import torch
import torch.nn.functional as func

In [18]:
with open("data/embeddings.p", "rb") as f:
    embeddings = pickle.load(f)

In [19]:
embeddings = torch.tensor(embeddings)

In [20]:
model = GraphEmbeddingEncoder(embeddings, emb_dim=500, compressed_dim=400, final_dim=50, embedding_dropout_pct=0.5)

In [21]:
model.load_state_dict(torch.load("saved_models/EmbeddingEncoder-ft_epoch40_dims400_50_embedding_pct0.5.pt"))

<All keys matched successfully>

In [22]:
device = torch.device("cuda")

In [23]:
model = model.to(device)

In [24]:
with open("data/artist_dict.p", "rb") as f:
    artist_dict = pickle.load(f)
with open("data/artist_names.p", "rb") as f:
    artist_names = pickle.load(f)

In [25]:
inputs = torch.tensor([i for i in range(len(artist_names))]).unsqueeze(1)

In [26]:
inputs = inputs.to(device)

In [27]:
final_embeddings = model(inputs)

In [28]:
final_embeddings.shape

torch.Size([5370, 50])

In [29]:
final_embeddings = func.normalize(final_embeddings, p=2, dim=1)

In [30]:
final_embeddings.shape

torch.Size([5370, 50])

In [31]:
transpose = final_embeddings.permute(1, 0)

In [32]:
cosine_similarities = torch.matmul(final_embeddings, transpose)

In [33]:
cosine_similarities = cosine_similarities.detach().cpu().numpy()

In [34]:
cosine_similarities.shape

(5370, 5370)

In [35]:
def get_top_n_indicies(simlarities, n=400):
    n = -1*(n+1)
    indicies = simlarities.argsort()[n:][::-1]
    return indicies

In [36]:
recommendations_per_artist_scores = {}
recommendations_per_artist_names = {}
for i, name in enumerate(artist_names):
    similarities = cosine_similarities[i]
    indicies = get_top_n_indicies(similarities)
    recommendations_per_artist_scores[name.lower()] = {}
    recommendations_per_artist_names[name.lower()] = []
    
    for index in indicies:
        if index != i:
            recommendations_per_artist_names[name.lower()].append(artist_names[index].lower())
            recommendations_per_artist_scores[name.lower()][artist_names[index].lower()] = similarities[index]

In [37]:
with open("data/recommendations_per_artist_scores.p", "wb") as f:
    pickle.dump(recommendations_per_artist_scores, f)

In [38]:
with open("data/recommendations_per_artist_names.p", "wb") as f:
    pickle.dump(recommendations_per_artist_names, f)